In [1]:
!pip install pandas openpyxl chromadb langchain sentence_transformers


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')


In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
import pandas as pd

df = pd.read_csv("/home/ibmuser01/src/legal/tala-web/Code/data/accident_datas.csv")
df

,Accident,Basic Fault,Adjustment Factors,Notes,Cases,URL,Laws
0,"신호기가 있는 횡단보도에서 녹색신호에 교차로를 통과한 차량(직진, 좌회전, 우회전 ...",도로교통법 제5조에 따라 보행자나 차마는 신호기의 신호에 따라야 할 의무가 있으므로...,① 야간 또는 주·정차된 차량 사이에서 보행자가 걸어 나오는 등 운전자가 보행자의 ...,⊙ 차량이 녹색신호에 교차로를 진입하여 보행자신호등 적색신호에 횡단보도를 건너고 있...,⊙대법원 1990.8.10. 선고 90도1116 판결 횡단보도의 표지판이나 신호대가...,https://drive.google.com/file/d/1MPFmVenWqLqjO...,"⊙도로교통법 제5조(신호 또는 지시에 따를 의무)\n① 도로를 통행하는 보행자, 차..."
1,"신호기가 있는 횡단보도에서 황색신호에 교차로를 통과한 차량(직진, 좌회전, 우회전 ...",차량이 황색신호에 진입한 경우 신호를 위반한 것이므로 보1 기준보다 보행자의 과실을...,① 야간 또는 주·정차된 차량 사이에서 보행자가 걸어 나오는 등 운전자가 보행자의 ...,NaN,⊙대법원 1990.8.10. 선고 90도1116 판결\n횡단보도의 표지판이나 신호대...,https://drive.google.com/file/d/1nsSoQyIPAaPy_...,"⊙도로교통법 제5조(신호 또는 지시에 따를 의무)\n① 도로를 통행하는 보행자, 차..."
2,"신호기가 있는 횡단보도에서 황색신호에 교차로를 통과한 차량(직진, 좌회전)이 보행자...",도로교통법 제5조에 따라 보행자나 차마 등은 신호기의 신호에 따라야 할 의무가 있으...,① 보행자가 보행신호등 녹색신호가 작동하자마자 횡단하는 경우로서 차량에게 예견 및 ...,녹색점멸신호는 녹색신호와 동일하게 본다(도로교통법 시행규칙 별표2 참조). 이하 동...,x,https://drive.google.com/file/d/1TpcR-6A2uD6HR...,"⊙도로교통법 제5조(신호 또는 지시에 따를 의무)\n① 도로를 통행하는 보행자, 차..."
3,"신호기가 있는 횡단보도에서 황색신호에 교차로에 통과한 차량(직진, 좌회전)이 보행자...",보행자가 녹색신호에 횡단을 개시하여 녹색신호에 충돌한 경우 보행자에게는 신호 위반의...,NaN,녹색점멸신호는 녹색신호와 동일하게 본다(도로교통법 시행규칙 별표3 참조). 이하 동...,서울지방법원 서부지원 1993.3.9. 선고 92가합12636 판결 야간에 신호기 ...,https://drive.google.com/file/d/1Glf3M1QtrCR0k...,"⊙도로교통법 제5조(신호 또는 지시에 따를 의무)\n① 도로를 통행하는 보행자, 차..."
4,"신호기가 있는 횡단보도에서 적색신호에 교차로를 통과한 차량(직진, 좌회전, 우회전\...",도로교통법 제5조에 따라 보행자나 차마 등은 신호기의 신호에 따라야 할 의무가 있고...,① 야간 또는 주·정차된 차량 사이에서 보행자가 걸어 나오는 등 운전자가 보행자의 ...,NaN,부산지방법원 2013.11.26. 선고 2011가단17263 판결 주간에 삼거리 교...,https://drive.google.com/file/d/1W6eo0EeZS4Rvw...,"⊙도로교통법 제5조(신호 또는 지시에 따를 의무)\n① 도로를 통행하는 보행자, 차..."
...,...,...,...,...,...,...,...
208,도로에서 정상 진행 중인 B차량과 도로를 횡단하는 A자전거가 충돌한 사고이다.,자전거 운전자가 도로를 횡단하고자 할 경우에는 자전거에서 내려서 횡단보도를 이용하거...,NaN,NaN,◆서울중앙지방법원 2007.6.27. 선고 2006가단183342 판결 비가 내리는...,https://drive.google.com/file/d/1ZPgksdemr2txb...,⊙도로교통법 제13조의2(자전거의 통행방법의 특례)\n③ 자전거의 운전자는 길가장자...
209,자전거횡단도를 횡단하는 A자전거와 직진하여 자전거횡단도를 통과하는 B차량이 충돌한\...,자전거는 도로교통법 제15조의 2 및 동법 시행규칙 [별표2]에 따라 자전거 횡단도...,NaN,NaN,◆전주지방법원 2007.10.12. 선고 2005가단25092 판결 주간에 편도3차...,https://drive.google.com/file/d/1_ekBBIhVXoQU4...,⊙도로교통법 제15조의2(자전거 횡단도의 설치 등)\n② 자전거 운전자가 자전거를 ...
210,자전거 전용도로에서 통행 중인 A자전거와 일반도로에서 자전거 전용도로로\n진로변경을...,전거 전용도로/차로는 자전거만 통행할 수 있도록 분리대 등의 시설물\n이나 차선 및...,NaN,NaN,전주지방법원 2010. 4. 16. 선고 2009가단22343 판결\n편도 5차로 ...,https://drive.google.com/file/d/1xxbmRfAJQkA2G...,\n⊙자전거 이용 활성화에 관한 법률 제3조(자전거도로의 구분) 자전거도로는 다음과...
211,자전거 전용차로에서 통행 중인 A자전거와 일반도로에서 자전거 전용차로로\n진로변경을...,전거 전용도로/차로는 자전거만 통행할 수 있도록 분리대 등의 시설물\n이나 차선 및...,NaN,NaN,x,https://drive.google.com/file/d/1EbbN9MsT7znvT...,⊙자전거 이용 활성화에 관한 법률 제3조(자전거도로의 구분) 자전거도로는 다음과 같...


In [4]:
# 1. 임베딩 모델 로드
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/tmp/ipykernel_489763/323747046.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/home/ibmuser01/src/gust-ibm-ai-hackathon/.venv/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# 2. 사고 상황 데이터 준비 (Accident 컬럼만 사용)
accident_texts = df["Accident"].tolist()
documents = [Document(page_content=text) for text in accident_texts]
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
import chromadb


In [6]:
# 3. 최신 ChromaDB 설정
persist_directory = "./chroma_accidents"
client_settings = chromadb.Settings(
    is_persistent=True,
    persist_directory=persist_directory
)

In [7]:
# 4. ChromaDB에 사고 상황 저장
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=persist_directory,
    client_settings=client_settings
)

In [8]:
# 4. 벡터 DB 저장
vectorstore.persist()

print("Chroma DB 생성 및 저장 완료!")
# 유사한 사고 사례 검색 함수
def search_similar_accidents(user_input, top_k=3):
    query_embedding = embeddings.embed_query(user_input)
    results = vectorstore.similarity_search_by_vector(query_embedding, k=top_k)
    return results

Chroma DB 생성 및 저장 완료!


/tmp/ipykernel_489763/1835085329.py:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [10]:
# 테스트 입력
user_input = "신호등이 없는 횡단보도에서 보행자를 충돌한 사고"
similar_cases = search_similar_accidents(user_input)

# 결과 출력
for idx, case in enumerate(similar_cases):
    print(f"🔹 사고 사례 {idx+1}:\n{case.page_content}\n")


🔹 사고 사례 1:
신호기가 있는 횡단보도 부근에서 우회전차 규제신호기 적색신호에 우회전하여 교차로
및 횡단보도를 통과한 차량이 횡단보도 부근을 건너고 있던 보행자를 충격한 사고이다.

🔹 사고 사례 2:
신호기가 있는 횡단보도 부근에서 우회전차 규제신호기 적색신호에 우회전하여 교차로
및 횡단보도를 통과한 차량이 횡단보도 부근을 건너고 있던 보행자를 충격한 사고이다.

🔹 사고 사례 3:
교차로에서 우회전하는 A차량과 유턴신호에 따라 유턴하는 B차량이 충돌한 사고이다.



In [12]:
# 검색된 사고 사례에서 추가 정보를 가져오는 함수
def get_accident_details(similar_cases, df):
    details = []
    for case in similar_cases:
        accident_text = case.page_content
        row = df[df["Accident"] == accident_text].iloc[0]  # 해당 사고 사례 찾기
        details.append({
            "Accident": row["Accident"],
            "Basic Fault": row["Basic Fault"],
            "Cases": row["Cases"],
            "Laws": row["Laws"],
            "URL": row["URL"],
        })
    return details

# 검색된 사고 사례에 대한 추가 정보 가져오기
accident_details = get_accident_details(similar_cases, df)

# 출력
for detail in accident_details:
    print(f"🔹 사고 상황: {detail['Accident']}")
    print(f"✅ 기본 과실: {detail['Basic Fault']}")
    print(f"📜 관련 판례: {detail['Cases']}")
    print(f"⚖ 관련 법규: {detail['Laws']}")
    print(f"🎥 동영상 링크: {detail['URL']}\n")


🔹 사고 상황: 신호기가 있는 횡단보도 부근에서 우회전차 규제신호기 적색신호에 우회전하여 교차로
및 횡단보도를 통과한 차량이 횡단보도 부근을 건너고 있던 보행자를 충격한 사고이다.
✅ 기본 과실: 우회전차량 규제신호기가 적색신호인 경우에는 차량이 신호를 위반한 것이나,
보행자도 횡단보도를 벗어나서 도로를 횡단한 과실이 있으므로 보행자신호가 녹색인
경우는 보행자의 기본 과실비율을 10%로 정하였다. 하지만 보행자신호가 적색인 경우
에는 보행자의 신호위반과 우회전차량의 과실을 고려하여 보행자의 기본 과실비율을
40%로 정한다.
📜 관련 판례: ◆대법원 1997.10.10. 선고 97도835 판결 교차로와 횡단보도가 인접하여 설치되어 있고 차량용 신호기는 교차로에만 설치된 경우에 있어서는 그 차량용 신호기는 차량에 대하여 교차로의 통행은 물론 교차로 직전의 횡단보도에 대한 통행까지도 아울러 지시하는 것이라고 보아야 할 것이고 횡단보도의 보행 등 측면에 차량보조등이 설치되어 있지 않다고 하여 횡단보도에 대한 차량용 신호등이 없는 상태라고는 볼 수 없다.
◆대법원 1994.8.23. 선고 94도199 판결 횡단보도의 양쪽 끝에 서로 마주보고 횡단보도의 통행인을 위한 보행자신호등이 각 설치되어 있고 그 신호등 측면에 차로진행방향을 향하여 종형 이색등신호기가 각각 별도로 설치되어 있다면, 종형 이색등신호기는 교차로를 통과하는 차마에 대한 진행방법을 지시하는 신호기라고 보는 것이 타당하다.
⚖ 관련 법규: ⊙도로교통법 제5조(신호 또는 지시에 따를 의무)
① 도로를 통행하는 보행자, 차마 또는 노면전차의 운전자는 교통안전시설이 표시하는 신호
또는 지시와 다음 각 호의 어느 하나에 해당하는 사람이 하는 신호 또는 지시를 따라야 한다.
1. 교통정리를 하는 경찰공무원(의무경찰을 포함한다. 이하 같다) 및 제주특별자치도의 자치경찰
공무원(이하 “자치경찰공무원”이라 한다)
2. 경찰공무원(자치경찰공무원을 포함한다. 이하 같다)을 보조하는 사람으로서 대통령령으로
정하는 사람(이하 “

In [13]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from langchain.llms import WatsonxLLM
load_dotenv()
# IBM Cloud 상의 watsonx SAS url은 provision된 region별로 정해져 있음. https://cloud.ibm.com/apidocs/watsonx-ai 참조.
project_id = os.getenv("PROJECT_ID", None)
wml_credentials = {
    "apikey": os.getenv("API_KEY", None),
    "url": 'https://us-south.ml.cloud.ibm.com'
}
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY.value,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 500,
    GenParams.STOP_SEQUENCES: ["<|endoftext|>"]
}

model_id =  'ibm/granite-3-8b-instruct' # 'mistralai/mistral-large' # 'meta-llama/llama-3-1-70b-instruct' # ModelTypes.LLAMA_2_70B_CHAT.value #
watsonx_llama2_korean = WatsonxLLM(
    model_id=model_id,
    url=wml_credentials.get("url"),
    apikey=wml_credentials.get("apikey"),
    project_id=project_id, 
    params=parameters
)

/tmp/ipykernel_489763/3561498226.py:28: LangChainDeprecationWarning: The class `WatsonxLLM` was deprecated in LangChain 0.0.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-ibm package and should be used instead. To use it run `pip install -U :class:`~langchain-ibm` and import as `from :class:`~langchain_ibm import WatsonxLLM``.
  watsonx_llama2_korean = WatsonxLLM(


In [14]:
from langchain.prompts import PromptTemplate
# 1. WatsonX 기반 프롬프트 템플릿 정의
prompt_template = PromptTemplate(
    input_variables=["accident", "basic_fault", "cases", "laws", "user_question"],
    template="""
    # 사고 사례 분석 시스템 
    # 아래 정보를 종합하여 법적 근거를 바탕으로 신뢰할 수 있는 답변을 제공해주세요. 

    - 사고 사례:
      {accident}

    - 기본 과실 설명:
      {basic_fault}

    - 관련 판례:
      {cases}

    - 관련 법규:
      {laws}

    - 사용자 질문:
      {user_question}

    
    """
)

In [15]:
def get_watsonx_response(selected_case, user_question):
    # 사고 데이터 조합
    prompt_text = prompt_template.format(
        accident=selected_case["Accident"],
        basic_fault=selected_case["Basic Fault"],
        cases=selected_case["Cases"],
        laws=selected_case["Laws"],
        user_question=user_question
    )

    # WatsonX 모델 호출 (리스트 형태로 전달)
    response = watsonx_llama2_korean.generate([prompt_text])  

    # 🔹 WatsonX 응답에서 텍스트 추출
    return response.generations[0][0].text  # LLMResult에서 실제 응답 추출


In [16]:
# 3. 테스트 실행 (사용자 질문 입력)
selected_case = {
    "Accident": "신호기가 있는 횡단보도 부근에서 우회전차 규제신호기 적색신호에 우회전하여 교차로 및 횡단보도를 통과한 차량이 횡단보도 부근을 건너고 있던 보행자를 충격한 사고이다.",
    "Basic Fault": "우회전차량 규제신호기가 적색신호인 경우에는 차량이 신호를 위반한 것이나,보행자도 횡단보도를 벗어나서 도로를 횡단한 과실이 있으므로 보행자신호가 녹색인 경우는 보행자의 기본 과실비율을 10%로 정하였다. 하지만 보행자신호가 적색인 경우에는 보행자의 신호위반과 우회전차량의 과실을 고려하여 보행자의 기본 과실비율을 40%로 정한다.",
    "Cases": "◆대법원 1997.10.10. 선고 97도835 판결 교차로와 횡단보도가 인접하여 설치되어 있고 차량용 신호기는 교차로에만 설치된 경우에 있어서는 그 차량용 신호기는 차량에 대하여 교차로의 통행은 물론 교차로 직전의 횡단보도에 대한 통행까지도 아울러 지시하는 것이라고 보아야 할 것이고 횡단보도의 보행 등 측면에 차량보조등이 설치되어 있지 않다고 하여 횡단보도에 대한 차량용 신호등이 없는 상태라고는 볼 수 없다. ◆대법원 1994.8.23. 선고 94도199 판결 횡단보도의 양쪽 끝에 서로 마주보고 횡단보도의 통행인을 위한 보행자신호등이 각 설치되어 있고 그 신호등 측면에 차로진행방향을 향하여 종형 이색등신호기가 각각 별도로 설치되어 있다면, 종형 이색등신호기는 교차로를 통과하는 차마에 대한 진행방법을 지시하는 신호기라고 보는 것이 타당하다.",
    "Laws": """⊙도로교통법 제5조(신호 또는 지시에 따를 의무)
① 도로를 통행하는 보행자, 차마 또는 노면전차의 운전자는 교통안전시설이 표시하는 신호
또는 지시와 다음 각 호의 어느 하나에 해당하는 사람이 하는 신호 또는 지시를 따라야 한다.
1. 교통정리를 하는 경찰공무원(의무경찰을 포함한다. 이하 같다) 및 제주특별자치도의 자치경찰
공무원(이하 “자치경찰공무원”이라 한다)
2. 경찰공무원(자치경찰공무원을 포함한다. 이하 같다)을 보조하는 사람으로서 대통령령으로
정하는 사람(이하 “경찰보조자”라 한다)
⊙도로교통법 제10조(도로의 횡단)
② 보행자는 제1항에 따른 횡단보도, 지하도, 육교나 그 밖의 도로 횡단시설이 설치되어 있는
도로에서는 그곳으로 횡단하여야 한다. 다만, 지하도나 육교 등의 도로 횡단시설을 이용할
수 없는 지체장애인의 경우에는 다른 교통에 방해가 되지 아니하는 방법으로 도로횡단
시설을 이용하지 아니하고 도로를 횡단할 수 있다.
⊙도로교통법 제27조(보행자의 보호)
① 모든 차 또는 노면전차의 운전자는 보행자(제13조의2제6항에 따라 자전거등에서 내려서
자전거등을 끌거나 들고 통행하는 자전거등의 운전자를 포함한다)가 횡단보도를 통행하고
있거나 통행하려고 하는 때에는 보행자의 횡단을 방해하거나 위험을 주지 아니하도록
그 횡단보도 앞(정지선이 설치되어 있는 곳에서는 그 정지선을 말한다)에서 일시정지
하여야 한다.
② 모든 차 또는 노면전차의 운전자는 교통정리를 하고 있는 교차로에서 좌회전이나 우회전을
하려는 경우에는 신호기 또는 경찰공무원등의 신호나 지시에 따라 도로를 횡단하는 보행
자의 통행을 방해하여서는 아니 된다.
③ 모든 차의 운전자는 교통정리를 하고 있지 아니하는 교차로 또는 그 부근의 도로를 횡단
하는 보행자의 통행을 방해하여서는 아니 된다.
⊙도로교통법 제48조(안전운전 및 친환경 경제운전의 의무)
① 모든 차 또는 노면전차의 운전자는 차 또는 노면전차의 조향장치와 제동장치, 그 밖의 장치를
정확하게 조작하여야 하며, 도로의 교통상황과 차 또는 노면전차의 구조 및 성능에 따라
다른 사람에게 위험과 장해를 주는 속도나 방법으로 운전하여서는 아니 된다.
⊙도로교통법 시행규칙 별표2(신호기가 표시하는 신호의 종류 및 신호의 뜻)"""
}

user_question = "나는 보행자인데 이 사고에서 내 책임 비율은 몇 %야?"
response = get_watsonx_response(selected_case, user_question)

# 결과 출력
print("🔹 AI 응답:", response)

🔹 AI 응답: 
사고 사례에 따르면 보행자의 기본 과실비율은 10%입니다. 하지만 보행자신호가 적색인 경우에는 보행자의 신호위반과 우회전차량의 과실을 고려하여 보행자의 기본 과실비율을 40%로 정합니다. 따라서 이 사고에서 보행자의 책임 비율은 40%입니다.
